# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>




## Aparajita Singh (apsi2875)
#### Resources: Class notes/lectures and the links provided.
#### Collaborators: Amit Baran Roy

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [3]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [4]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [5]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [6]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [7]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [8]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   null|     BE|   null|    null|      1|  null|   269|  6|    69| null|       1|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|3070802| 1963| 1096|   null|     US|     TX|    null|      1|  null|     2|  6|    63| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|
|3070803| 1963| 1096|   null|     US|     IL|    null|      1|  null|     2|  6|    6

#### Create another table patents_info with only the patent number and the patent state.

In [9]:
patents_info = patents.select("PATENT", "POSTATE")
patents_info.show(20)

+-------+-------+
| PATENT|POSTATE|
+-------+-------+
|3070801|   null|
|3070802|     TX|
|3070803|     IL|
|3070804|     OH|
|3070805|     CA|
|3070806|     PA|
|3070807|     OH|
|3070808|     IA|
|3070809|     AZ|
|3070810|     IL|
|3070811|     CA|
|3070812|     LA|
|3070813|     NY|
|3070814|     MN|
|3070815|     CO|
|3070816|     OK|
|3070817|     RI|
|3070818|     IN|
|3070819|     TN|
|3070820|   null|
+-------+-------+
only showing top 20 rows



#### Create aliases for the citations table and the patents_info table.

In [10]:
cita = citations.alias('cita')
pi = patents_info.alias('pi')

### Join the citations table and the patents_info table, with the help of their aliases created in previous step. For joining, use "CITING" column from citations and "PATENT" column from patents_info for comparison.

In [11]:
joined_table = cita.join(pi, cita.CITING == pi.PATENT)
joined_table.show()

+-------+-------+-------+-------+
| CITING|  CITED| PATENT|POSTATE|
+-------+-------+-------+-------+
|3858258|1331793|3858258|     CA|
|3858258|1540798|3858258|     CA|
|3858527| 924225|3858527|   null|
|3858527|2444326|3858527|   null|
|3858527|2705120|3858527|   null|
|3858527|2967080|3858527|   null|
|3858527|3602157|3858527|   null|
|3858527|3638586|3858527|   null|
|3858527|3699902|3858527|   null|
|3858560| 957631|3858560|     IN|
|3858597|3675252|3858597|     MT|
|3858597|3815160|3858597|     MT|
|3858770|2290722|3858770|     CA|
|3858770|2777621|3858770|     CA|
|3858770|2782969|3858770|     CA|
|3858770|3040941|3858770|     CA|
|3859029| 982044|3859029|     NY|
|3859029|1020004|3859029|     NY|
|3859029|1830227|3859029|     NY|
|3859029|2752631|3859029|     NY|
+-------+-------+-------+-------+
only showing top 20 rows



#### Create a new table by selecting CITED, CITING and POSTATE from joined_table. Rename "POSTATE" column to "CITING_STATE". Renaming is essential for the next steps.

In [12]:
table1 = joined_table.select("CITED", "CITING", "POSTATE")
table1 = table1.withColumnRenamed("POSTATE", "CITING_STATE")
table1.show(10)

+-------+-------+------------+
|  CITED| CITING|CITING_STATE|
+-------+-------+------------+
|1331793|3858258|          CA|
|1540798|3858258|          CA|
| 924225|3858527|        null|
|2444326|3858527|        null|
|2705120|3858527|        null|
|2967080|3858527|        null|
|3602157|3858527|        null|
|3638586|3858527|        null|
|3699902|3858527|        null|
| 957631|3858560|          IN|
+-------+-------+------------+
only showing top 10 rows



#### Create an alias for table1. Perform join on table1 and patents_info. Display the table obtained.

In [13]:
t1 = table1.alias('t1')

joined_table1 = t1.join(pi, t1.CITED == pi.PATENT)
joined_table1.show()

+-------+-------+------------+-------+-------+
|  CITED| CITING|CITING_STATE| PATENT|POSTATE|
+-------+-------+------------+-------+-------+
|3071083|3951073|          WA|3071083|   null|
|3071083|4401034|        null|3071083|   null|
|3071083|4802416|        null|3071083|   null|
|3071452|4192656|          PA|3071452|   null|
|3071452|4045189|        null|3071452|   null|
|3071753|4496943|          NJ|3071753|     MN|
|3071753|4345315|          TN|3071753|     MN|
|3071753|4120573|          IL|3071753|     MN|
|3071753|3949375|          NJ|3071753|     MN|
|3071753|4271479|          NY|3071753|     MN|
|3071941|5062619|        null|3071941|   null|
|3072100|4280448|          KS|3072100|     IL|
|3072100|3861359|          IL|3072100|     IL|
|3072100|4138968|          KS|3072100|     IL|
|3072100|3894516|          OK|3072100|     IL|
|3072100|4572109|          SC|3072100|     IL|
|3072274|4396343|          OH|3072274|     WI|
|3072274|4955781|          OR|3072274|     WI|
|3072274|4907

#### Create another table by selecting CITED, POSTATE, CITING and CITING_STATE. Rename the "POSTATE" column to "CITED_STATE". The required intermediate table is obtained now.

In [14]:
table2 = joined_table1.select("CITED", "POSTATE", "CITING", "CITING_STATE")
table2 = table2.withColumnRenamed("POSTATE", "CITED_STATE")
table2.show(10)

+-------+-----------+-------+------------+
|  CITED|CITED_STATE| CITING|CITING_STATE|
+-------+-----------+-------+------------+
|3071083|       null|3951073|          WA|
|3071083|       null|4401034|        null|
|3071083|       null|4802416|        null|
|3071452|       null|4192656|          PA|
|3071452|       null|4045189|        null|
|3071753|         MN|4496943|          NJ|
|3071753|         MN|4345315|          TN|
|3071753|         MN|4120573|          IL|
|3071753|         MN|3949375|          NJ|
|3071753|         MN|4271479|          NY|
+-------+-----------+-------+------------+
only showing top 10 rows



#### Rearrange the table and display it.

In [15]:
table3 = table2.select("CITING", "CITING_STATE", "CITED", "CITED_STATE")
table3.show(10)

+-------+------------+-------+-----------+
| CITING|CITING_STATE|  CITED|CITED_STATE|
+-------+------------+-------+-----------+
|3951073|          WA|3071083|       null|
|4401034|        null|3071083|       null|
|4802416|        null|3071083|       null|
|4192656|          PA|3071452|       null|
|4045189|        null|3071452|       null|
|4496943|          NJ|3071753|         MN|
|4345315|          TN|3071753|         MN|
|4120573|          IL|3071753|         MN|
|3949375|          NJ|3071753|         MN|
|4271479|          NY|3071753|         MN|
+-------+------------+-------+-----------+
only showing top 10 rows



#### Create an alias for table3.

In [16]:
table3.show(10)
t3 = table3.alias('t3')

+-------+------------+-------+-----------+
| CITING|CITING_STATE|  CITED|CITED_STATE|
+-------+------------+-------+-----------+
|3951073|          WA|3071083|       null|
|4401034|        null|3071083|       null|
|4802416|        null|3071083|       null|
|4192656|          PA|3071452|       null|
|4045189|        null|3071452|       null|
|4496943|          NJ|3071753|         MN|
|4345315|          TN|3071753|         MN|
|4120573|          IL|3071753|         MN|
|3949375|          NJ|3071753|         MN|
|4271479|          NY|3071753|         MN|
+-------+------------+-------+-----------+
only showing top 10 rows



#### From the intermediate table, extract those rows where CITING_STATE and CITED_STATE are equal (required problem). Also, filter out the rows where CITING_STATE or CITED_STATE are equal to null.

In [17]:
table4 = t3.filter(t3["CITING_STATE"]==t3["CITED_STATE"]).filter(t3["CITING_STATE"]!="null").filter(t3["CITED_STATE"]!="null")
table4.show(20)

+-------+------------+-------+-----------+
| CITING|CITING_STATE|  CITED|CITED_STATE|
+-------+------------+-------+-----------+
|4383847|          AL|3177062|         AL|
|3878730|          AL|3610053|         AL|
|3877317|          AL|3610053|         AL|
|4601891|          AL|3733191|         AL|
|4237106|          AL|3733191|         AL|
|4028087|          AL|3733191|         AL|
|4427432|          AL|3969483|         AL|
|4246248|          AL|3969483|         AL|
|4090893|          AL|3974004|         AL|
|4429634|          AL|3974004|         AL|
|4471988|          AL|4415080|         AL|
|4725766|          AL|4546264|         AL|
|4730154|          AL|4546264|         AL|
|4735382|          AL|4546264|         AL|
|5049030|          AL|4547115|         AL|
|4880701|          AL|4698246|         AL|
|4806399|          AL|4698246|         AL|
|5213858|          AL|4698246|         AL|
|4888222|          AL|4698246|         AL|
|4983431|          AL|4698246|         AL|
+-------+--

#### Create an alias for table4. Use groupBy by CITING to obtain the number of same state citations. Use count() to count the rows in each group obtained. Rename the column count to SAME_STATE.

t4 = table4.alias('t4')
table5 = t4.groupBy("CITING").count()
table5 = table5.withColumnRenamed("count", "SAME_STATE")
table5.show(20)

#### Create aliases for table5 and patents table.

In [19]:
pa = patents.alias('pa')
t5 = table5.alias('t5')

#### As a final step, join table5 with the original table patents (with all the information) to output the desired result. Display the table in descending order by doing ascending=False. 

In [20]:
pa.join(t5, t5.CITING==pa.PATENT).sort('SAME_STATE', ascending=False).show(20)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+----------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD| CITING|SAME_STATE|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+----------+
|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  null|   326|  4|    46|  159|       0|     1.0|   null|  0.6186|    null|  4.8868|  0.0455|   0.044|    null|    null|5959466|       125|
|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  null|   114|  5|    55|  200|       0|   0.995|   null|  0.7201|    null|   12.45|     0.0|     0.0|    null|    null|5983822|  